In [3]:
import pandas as pd
from scipy.stats import describe
import numpy as np

from hurst import compute_Hc, random_walk
from scipy import stats
import os


In [4]:
from SMC_ABC_init import all_summary_stats

In [5]:
df = pd.read_csv("C:\My Work Documents\Dissertation\pythonCodeVersions\out2.csv")
df = pd.DataFrame(df)
df.shape
df.columns = range(500)

In [6]:
p_true = pd.read_csv("C:\My Work Documents\Dissertation\pythonCodeVersions\p_true.csv")
p_true = pd.DataFrame(p_true)

In [7]:
def extract_col(x):
    if min(x)>0:
        return True
    else:
        return False

In [8]:
df_pos = df.loc[:, df.apply(extract_col, axis = 0)]
df_pos = np.log(df_pos)
df_pos.columns = range(0,df_pos.shape[1])

In [9]:
param = pd.read_csv("C:\My Work Documents\Dissertation\pythonCodeVersions\param2.csv")
param = param.transpose()
param.columns = range(500)
param_pos = param.loc[:, df.apply(extract_col, axis = 0)]

In [10]:
param_pos

,0,1,2,3,4,6,7,8,9,10,...,490,491,492,493,494,495,496,497,498,499
delta,0.021013,0.031615,0.002215,0.063729,0.091807,0.035743,0.076862,0.019197,0.077130,0.031622,...,0.094374,0.051913,0.061277,0.095374,0.058241,0.053520,0.047424,0.057419,0.062327,0.071575
mu,0.052332,0.051754,0.000875,0.029647,0.081560,0.064384,0.054222,0.041693,0.024302,0.008406,...,0.065526,0.065682,0.016303,0.007585,0.051888,0.059198,0.076994,0.095419,0.076679,0.071761
alpha,0.471739,0.190073,0.189818,0.231375,0.307359,0.184073,0.496707,0.382793,0.132031,0.153539,...,0.394588,0.272863,0.188345,0.306717,0.478520,0.224248,0.467194,0.358351,0.397837,0.469465
lambda_0,105.160987,104.736862,8.312104,116.620110,2.940078,89.255561,63.756305,14.440517,76.808782,71.900131,...,170.828723,142.203124,81.045455,131.855002,41.442387,16.236464,9.014965,188.539904,74.421893,172.680085
C_lambda,13.354103,15.555103,6.425772,19.812143,0.321699,4.736352,19.678950,17.150366,1.448160,7.863772,...,15.359609,2.255957,3.795345,14.431163,5.303501,7.476019,12.511655,10.336434,13.156876,3.028492
delta_S,0.033766,0.003732,0.004276,0.021337,0.046633,0.036377,0.074613,0.051222,0.024036,0.038115,...,0.091790,0.002456,0.056583,0.065410,0.049509,0.051215,0.037093,0.017096,0.016750,0.005393


In [11]:
def summary_stats_extra(x):
    """outputs additional summary statistics: skewness, kurtosis, Hurst"""

    try:
        H, c, data = compute_Hc(x, kind='price', simplified=True)
    except Exception as e:
        H = 0.25
        print(e)
        
    # count, mean, std, min, 25%, 50%, 75%, max
    s1 = x.describe()
    
    # Kolmogorov Smirnov 2 sample test statistic (if 0 - identical)
    ks_stat = {"KS": stats.ks_2samp(np.ravel(x), np.ravel(p_true))[0]}
    
    return {"mean": s1.loc["mean"],
            "std": s1.loc["std"],
            "skew": x.skew(),
            "kurt": x.kurt(),
            "hurst": H,
            **ks_stat}



In [12]:
SS1 = summary_stats_extra(df_pos.loc[1])
#SS = df_pos.apply(all_summary_stats, price_obs=p_true, axis = 0)
#SS1_df = pd.DataFrame(SS1[0])
SS1_df = pd.DataFrame(list(SS1.items()), columns=['SS_name', 'SS'])
SS1_df

,SS_name,SS
0,mean,4.605171
1,std,0.000015
2,skew,1.040604
3,kurt,9.058074
4,hurst,0.292721
5,KS,0.713988


In [21]:
results, SS2 = pd.cut(df_pos.loc[1], bins=9, retbins=True)
SS2_name = np.array(range(21)) 
SS2_df = pd.DataFrame(np.hstack((SS2_name.T[:,None],SS2[:,None])), columns = ['SS_name', 'SS'])
SS2_df

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [14]:
dfs = [SS1_df, SS2_df]
dfs = pd.concat(dfs)
dfs["SS"][:,None]

array([[4.60517149e+00],
       [1.45148738e-05],
       [1.04060375e+00],
       [9.05807422e+00],
       [2.92720920e-01],
       [7.13987928e-01],
       [4.60510002e+00],
       [4.60510818e+00],
       [4.60511618e+00],
       [4.60512418e+00],
       [4.60513218e+00],
       [4.60514018e+00],
       [4.60514818e+00],
       [4.60515618e+00],
       [4.60516418e+00],
       [4.60517218e+00],
       [4.60518018e+00],
       [4.60518818e+00],
       [4.60519618e+00],
       [4.60520418e+00],
       [4.60521218e+00],
       [4.60522018e+00],
       [4.60522818e+00],
       [4.60523618e+00],
       [4.60524418e+00],
       [4.60525218e+00],
       [4.60526018e+00]])

In [25]:
def summarise_price_path(price_path, islog = False):
    
    # log prices if not yet logged
    if islog == False:
        price_path = np.log(price_path)
        s
    SS1 = summary_stats_extra(price_path)
    SS1_df = pd.DataFrame(list(SS1.items()), columns=['SS_name', 'SS'])
    
    results, SS2 = pd.cut(price_path, bins=9, retbins=True)
    SS2_name = np.array(range(10)) 
    SS2_df = pd.DataFrame(np.hstack((SS2_name.T[:,None],SS2[:,None])), columns = ['SS_name', 'SS'])
    
    dfs = [SS1_df, SS2_df]
    dfs = pd.concat(dfs)
    
    arr = dfs["SS"]
    
    return(arr)
    
        

In [23]:
pd.DataFrame(p_true)

,0
0,4.605185
1,4.605190
2,4.605185
3,4.605190
4,4.605220
...,...
245,4.630438
246,4.629980
247,4.630336
248,4.629585


In [26]:
SS_true = p_true.apply(summarise_price_path, islog =True, axis = 0)
SS_true

,0
0,4.614581
1,0.011211
2,0.392750
3,-1.566143
4,0.674491
5,0.000000
0,4.599979
1,4.603642
2,4.607272
3,4.610902


In [27]:
SS = df_pos.apply(summarise_price_path, islog =True, axis = 0)


In [28]:
SS

,0,1,2,3,4,5,6,7,8,9,...,487,488,489,490,491,492,493,494,495,496
0,4.602150,4.622447,4.605175e+00,4.591404,4.605141,4.622402,4.603485,4.605359,4.602525,4.604753,...,4.671990,4.603873,4.604730,4.605748,4.606427,4.605382,4.604279,4.597053,4.606369,4.602337
1,0.001451,0.011551,1.042569e-07,0.013623,0.000034,0.007850,0.001597,0.000093,0.001554,0.000350,...,0.028289,0.000863,0.000317,0.000455,0.001436,0.000685,0.000297,0.020670,0.011254,0.003093
2,0.362726,0.293717,0.000000e+00,-0.162986,-0.147293,-1.018841,-0.103546,-0.486220,0.321011,-0.484101,...,-0.931331,-0.552512,0.213438,-0.146245,0.412727,0.204059,0.698197,-0.589190,-0.846656,-1.667319
3,-0.912945,-1.342560,0.000000e+00,-1.279148,-0.775569,-0.448543,-1.045040,-0.442798,-1.281681,-0.435724,...,-0.154635,0.321611,-0.875222,-0.067438,-1.649059,0.245527,0.002738,-0.701947,-0.807142,2.307296
4,0.492375,0.507060,5.073293e-01,0.614173,0.513378,0.631409,0.508812,0.426595,0.570434,0.469128,...,0.741147,0.361775,0.423073,0.523315,0.647655,0.567666,0.591614,0.543159,0.634116,0.532539
5,0.724087,0.449478,7.400000e-01,0.666957,0.720000,0.411130,0.594522,0.676000,0.702783,0.679739,...,0.852174,0.687565,0.678435,0.580000,0.547130,0.566783,0.744000,0.503565,0.360000,0.662522
0,4.599397,4.603439,4.605170e+00,4.568928,4.605065,4.604883,4.599492,4.605075,4.600233,4.603878,...,4.604981,4.600977,4.604008,4.604683,4.604601,4.603871,4.603778,4.550445,4.583649,4.591837
1,4.600115,4.607885,4.605171e+00,4.573928,4.605082,4.607917,4.600340,4.605127,4.600840,4.604066,...,4.617226,4.601521,4.604178,4.604923,4.605057,4.604275,4.603934,4.559302,4.587823,4.593477
2,4.600826,4.612290,4.605171e+00,4.578883,4.605099,4.610924,4.601181,4.605180,4.601442,4.604253,...,4.629362,4.602060,4.604346,4.605160,4.605509,4.604676,4.604088,4.568080,4.591959,4.595101
3,4.601537,4.616696,4.605172e+00,4.583839,4.605115,4.613931,4.602022,4.605232,4.602044,4.604440,...,4.641497,4.602599,4.604515,4.605398,4.605961,4.605077,4.604243,4.576858,4.596096,4.596726


In [29]:
SS.to_csv(os.path.join("C:\My Work Documents\Dissertation\pythonCodeVersions", 'SS_options.csv'), index=False)
param_pos.to_csv(os.path.join("C:\My Work Documents\Dissertation\pythonCodeVersions", 'param_options.csv'), index=False)
SS_true.to_csv(os.path.join("C:\My Work Documents\Dissertation\pythonCodeVersions", 'SS_true_options.csv'), index=False)
